<a href="https://colab.research.google.com/github/eunsinchoi/prectice01/blob/main/%ED%8C%8C%EC%9D%B4%ED%86%A0%EC%B9%98_%EB%B9%A0%EB%A5%B8_%EC%8B%9C%EC%9E%91.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch

In [2]:
from torch import nn
from torch.utils.data import DataLoader
from torchvision.transforms import ToTensor
from torchvision import datasets

In [3]:
training_data = datasets.FashionMNIST(
    root='data',
    train=True,
    download=True,
    transform=ToTensor(),
)

test_data = datasets.FashionMNIST(
    root='data',
    train=False,
    download=True,
    transform=ToTensor(),
)

100%|██████████| 26421880/26421880 [00:01<00:00, 19161132.16it/s]


Extracting data/FashionMNIST/raw/train-images-idx3-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 29515/29515 [00:00<00:00, 334554.54it/s]


Extracting data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 4422102/4422102 [00:00<00:00, 6168613.86it/s]


Extracting data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 5148/5148 [00:00<00:00, 15970619.08it/s]

Extracting data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to data/FashionMNIST/raw



In [4]:
batch_size = 64

train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader :
  print(f"Shape of X [N, C, H, w] : {X.shape}")
  print(f"Shape of y : {y.shape} {y.dtype}")
  break

Shape of X [N, C, H, w] : torch.Size([64, 1, 28, 28])
Shape of y : torch.Size([64]) torch.int64


In [5]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)

print(f"Using {device} device")

Using cpu device


In [6]:
class NeuralNetwork(nn.Module) :
  def __init__(self) :
    super().__init__()
    self.flatten = nn.Flatten()
    self.linear_relu_stack = nn.Sequential(
        nn.Linear(28*28, 512),
        nn.ReLU(),
        nn.Linear(512, 512),
        nn.ReLU(),
        nn.Linear(512,18)
    )

  def forward(self, x) :
    x = self.flatten(x)
    logits = self.linear_relu_stack(x)
    return logits

In [7]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=18, bias=True)
  )
)


In [8]:
# 모델 매개변수 최적화하기

loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In [9]:
def train(dataloader, model, loss_fn, optimizer):
  size = len(dataloader.dataset)
  for batch, (X, y) in enumerate(dataloader):
    X, y = X.to(device), y.to(device)

    #예측오류계산
    pred = model(X)
    loss = loss_fn(pred, y)

    #역전파
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if batch % 100 == 0 :
      loss, current = loss.item(), (batch+1)*len(X)
      print(f"loss : {loss:>7f} [{current:>5d}/{size:>5d}]")


In [10]:
def test(dataloader, model, loss_fn):
  size = len(dataloader.dataset)
  num_batches = len(dataloader)
  model.eval()
  test_loss, correct = 0, 0
  with torch.no_grad():
    for X, y in dataloader:
      X, y = X.to(device), y.to(device)
      pred = model(X)
      test_loss += loss_fn(pred, y).item()
      correct += (pred.argmax(1) == y).type(torch.float).sum().item()
      test_loss /= num_batches
      correct /= size
      print(f"Test Error : \n Accuray : {(100*correct):>0.1f}%, Avg loss : {test_loss:>8f} \n")

In [11]:
epochs = 5
for t in range(epochs):
  print(f"Epoch {t+1}\n----------------------------------")
  train(train_dataloader, model, loss_fn, optimizer)
  test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
----------------------------------
loss : 2.887555 [   64/60000]
loss : 2.845472 [ 6464/60000]
loss : 2.789972 [12864/60000]
loss : 2.758964 [19264/60000]
loss : 2.681506 [25664/60000]
loss : 2.606761 [32064/60000]
loss : 2.598839 [38464/60000]
loss : 2.498896 [44864/60000]
loss : 2.431158 [51264/60000]
loss : 2.363916 [57664/60000]
Test Error : 
 Accuray : 0.2%, Avg loss : 0.014924 

Test Error : 
 Accuray : 0.2%, Avg loss : 0.014756 

Test Error : 
 Accuray : 0.2%, Avg loss : 0.014594 

Test Error : 
 Accuray : 0.2%, Avg loss : 0.014877 

Test Error : 
 Accuray : 0.2%, Avg loss : 0.014859 

Test Error : 
 Accuray : 0.1%, Avg loss : 0.014670 

Test Error : 
 Accuray : 0.2%, Avg loss : 0.014624 

Test Error : 
 Accuray : 0.2%, Avg loss : 0.014688 

Test Error : 
 Accuray : 0.2%, Avg loss : 0.014959 

Test Error : 
 Accuray : 0.2%, Avg loss : 0.014916 

Test Error : 
 Accuray : 0.2%, Avg loss : 0.014639 

Test Error : 
 Accuray : 0.1%, Avg loss : 0.015101 

Test Error : 
 Accura

In [12]:
torch.save(model.state_dict(), "model.pth")
print("Saved PyTorch Model State to model.pth")

Saved PyTorch Model State to model.pth


In [13]:
model = NeuralNetwork().to(device)
model.load_state_dict(torch.load("model.pth"))

<All keys matched successfully>

In [14]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval()
x, y = test_data[0][0], test_data[0][1]
with torch.no_grad():
    x = x.to(device)
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')

Predicted: "Ankle boot", Actual: "Ankle boot"
